In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from datetime import datetime
import mysql.connector

db_source = r'C:\xampp\htdocs\MIQ_projects\cedur\db\cedur_3.sqlite3'
conn_source = sqlite3.connect(db_source)
conn_source.row_factory = sqlite3.Row 
cursor_src = conn_source.cursor()

mysql_conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="cedur",
    charset="utf8mb4",     
    use_unicode=True       
)
cursor = mysql_conn.cursor(dictionary=True)

In [43]:
# Diagnose

if 1 == 0 : 

    query ="SELECT DISTINCT fcid FROM `forms_10003`"    
    cursor.execute(query)
    rows = cursor.fetchall()
    base_user = {}
    for row in rows:
        base_user[row['fcid']] = {}

    query ="SELECT fcid,fid,fcont, usergroup FROM `forms_10003` WHERE fid in (95,96)"  
    cursor.execute(query)
    rows = cursor.fetchall()  
    for row in rows:
        base_user[row['fcid']][row['fid']] = row['fcont']
        base_user[row['fcid']]['usergroup'] = row['usergroup']

    leere_werte = []
    for fcid, fcont_a in base_user.items():
        if 95 not in fcont_a or fcont_a.get(95) in ('', None):
            leere_werte.append(fcid)
    # to fcid_str
    zahlen_als_strings = map(str, leere_werte)
    joined_string = ', '.join(zahlen_als_strings)
    fcid_str = f"({joined_string})"

    pid_a = {}
    query ="SELECT fcid, fcont, usergroup FROM `forms_10003` WHERE fid=102 AND fcid IN " + fcid_str 
    cursor.execute(query)
    rows = cursor.fetchall()  
    for row in rows:
        pid_a[row['fcont']] = {'fcid': row['fcid'], 'usergroup': row['usergroup'], '95':''}


    for pid in pid_a:    
        query ="SELECT pid, CDAI001 FROM `CEDUR3_allgemein` WHERE pid='" + pid + "' AND CDAI001<>''" 
        cursor_src.execute(query)
        rows = cursor_src.fetchall()  
        for row in rows:
            pid_a[row['pid']]['95'] = row['CDAI001']

    # Missing
    for pid in pid_a:
        if pid_a[pid]['95']=='':
            print(pid, pid_a[pid])

    # ADD/ REPLACE Dignosis
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    for pid, fcont_a in pid_a.items():
        if fcont_a['95'] != '':
            print(f"REPLACE INTO forms_10003 (fcid, fid, fcont, usergroup, muid, mts) VALUES ({fcont_a['fcid']}, 95, '{fcont_a['95']}', {fcont_a['usergroup']}, 0, '{ts}');")    

In [44]:
# Groesse

if 1 == 0 : 

    # missing = 95
    # cedur_name = 'CDAI001'
    missing = 97
    cedur_name = 'CDAI_Size'

    query ="SELECT DISTINCT fcid FROM `forms_10003`"    
    cursor.execute(query)
    rows = cursor.fetchall()
    base_user = {}
    for row in rows:
        base_user[row['fcid']] = {}
    print("Got DISTINCT fcid")

    query = "SELECT fcid,fid,fcont, usergroup FROM `forms_10003` WHERE fid in (" + str(missing) + ")"  
    cursor.execute(query)
    rows = cursor.fetchall()  
    for row in rows:
        base_user[row['fcid']][row['fid']] = row['fcont']
        base_user[row['fcid']]['usergroup'] = row['usergroup']
    print("Add fcid to missing:" + str(missing))


    leere_werte = []
    for fcid, fcont_a in base_user.items():
        if missing not in fcont_a or fcont_a.get(missing) in ('', None):
            leere_werte.append(fcid)
    # to fcid_str
    zahlen_als_strings = map(str, leere_werte)
    joined_string = ', '.join(zahlen_als_strings)
    fcid_str = f"({joined_string})"
    print(f"Missing: {len(leere_werte)}")
    
    pid_a = {}
    query ="SELECT fcid, fcont, usergroup FROM `forms_10003` WHERE fid=102 AND fcid IN " + fcid_str 
    cursor.execute(query)
    rows = cursor.fetchall()  
    for row in rows:
        pid_a[row['fcont']] = {'fcid': row['fcid'], 'usergroup': row['usergroup'], str(missing):''}
    print("pid_a on exisiting MIQ-patients build")

    for pid in pid_a:    
        query ="SELECT pid, " + cedur_name + " FROM `CEDUR3_allgemein` WHERE pid='" + pid + "' AND " + cedur_name + "<>''" 
        cursor_src.execute(query)
        rows = cursor_src.fetchall()  
        for row in rows:
            pid_a[row['pid']][str(missing)] = row[cedur_name]
    print("pid_a CEDUR values added")

    # Missing
    # for pid in pid_a:
    #     if pid_a[pid]['10003020']=='':
    #         print(pid, pid_a[pid])

    # ADD/ REPLACE Dignosis
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    for pid, fcont_a in pid_a.items():
        if fcont_a[str(missing)] != '':
            print(f"REPLACE INTO forms_10003 (fcid, fid, fcont, usergroup, muid, mts) VALUES ({fcont_a['fcid']}, {missing}, '{fcont_a[str(missing)]}', {fcont_a['usergroup']}, 0, '{ts}');")    

In [45]:
# Erstdiagnose

if 1 == 1 : 

    # missing = 95
    # cedur_name = 'CDAI001'
    missing = 102200
    cedur_name = 'DatumErstdiagnose'

    query ="SELECT DISTINCT fcid FROM `forms_10010` WHERE fid in (" + str(missing) + ") AND muid < 0 AND fcont<>''"    
    cursor.execute(query)
    rows = cursor.fetchall()
    fcid_a = {}
    for row in rows:
        fcid_a[row['fcid']] = {}
    print("Got DISTINCT fcids:", len(fcid_a))
    
    query = "SELECT fcid,fid,fcont, usergroup FROM `forms_10010` WHERE fid = " + str(missing) + " AND muid < 0 AND fcont<>''"  
    cursor.execute(query)
    rows = cursor.fetchall()  
    for row in rows:
        fcid_a[row['fcid']][row['fid']] = row['fcont']
        fcid_a[row['fcid']]['usergroup'] = row['usergroup']
    print("Add fcid to missing")
    
    fcid_cedur_a = {}
    query ="SELECT a.*, i.fcid FROM CEDUR3_allgemein AS a LEFT JOIN CEDUR3_Info AS i ON a.cid = i.cid WHERE " + cedur_name + "<>'';"
    cursor_src.execute(query)
    rows = cursor_src.fetchall()  
    for row in rows:
        fcid_cedur_a[int(row['fcid'])] = row[cedur_name] 
    print("Cedur read fcids:", len(fcid_cedur_a))
    
    for fcid in fcid_cedur_a:
        if fcid not in fcid_a:
            print(fcid, fcid_cedur_a[fcid])

        
        # print("Missing:", fcid, fcid_cedur_a[fcid])
       


    # # ADD/ REPLACE Dignosis
    # ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # for pid, fcont_a in pid_a.items():
    #     if fcont_a[str(missing)] != '':
    #         print(f"REPLACE INTO forms_10003 (fcid, fid, fcont, usergroup, muid, mts) VALUES ({fcont_a['fcid']}, {missing}, '{fcont_a[str(missing)]}', {fcont_a['usergroup']}, 0, '{ts}');")   

Got DISTINCT fcids: 27758
Add fcid to missing
Cedur read fcids: 27775
2016052400218371 2015-02
2013081300225952 2005
2013081200225979 2008
2013080700226004 2005
2013080500226036 2009
2013080400226051 1993
2013071700226185 2006
2013071500226213 1989
2013071100226218 2010-09
2013062500226260 1999
2025081800300021 2017
2025081800300023 2021
2024073000304203 2022-06
2024011500305335 2020-03
2024010700305359 2024-01-02
2023103000305536 2020-03
2023091900305621 2021-09


In [46]:
cursor_src.close()
conn_source.close()
cursor.close()
mysql_conn.close()